In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import sys
sys.path.append('/content/drive/My Drive/685/Sentence-Simplification-using-BERT-GPT2/')
%cd '/content/drive/My Drive/685/Sentence-Simplification-using-BERT-GPT2/'

/content/drive/My Drive/685/Sentence-Simplification-using-BERT-GPT2


In [11]:
!pip install -q transformers==4.17.0 datasets==2.0.0 rich[jupyter]
!pip install -q googletrans==3.1.0a0
!pip install -q -U PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
print('success!')

     |████████████████████████████████| 3.8 MB 5.2 MB/s 
     |████████████████████████████████| 325 kB 46.8 MB/s 
     |████████████████████████████████| 231 kB 52.9 MB/s 
     |████████████████████████████████| 1.1 MB 42.1 MB/s 
     |████████████████████████████████| 212 kB 46.9 MB/s 
     |████████████████████████████████| 136 kB 54.5 MB/s 
     |████████████████████████████████| 127 kB 5.0 MB/s 
     |████████████████████████████████| 271 kB 51.6 MB/s 
     |████████████████████████████████| 144 kB 63.1 MB/s 
     |████████████████████████████████| 94 kB 2.0 MB/s 
     |████████████████████████████████| 51 kB 2.4 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
success!


In [12]:
import googletrans
#print(googletrans.LANGUAGES)
from googletrans import Translator
translator = translator = Translator()

In [13]:
import string
translate_table = dict((ord(char), None) for char in string.punctuation)   

NUM_EXAMPLES = 100
train_examples = []
file_path = 'dataset/src_train.txt'

with open(file_path, 'r', encoding="utf8") as f:
    sents = [next(f) for x in range(NUM_EXAMPLES)]
    for s in sents:
        train_examples.append(s.strip().translate(translate_table))

In [14]:
train_examples_augmented = []

for example in train_examples:
    train_examples_augmented.append(example) # always include the original example
    latin_languages = ['de', 'fr', 'it', 'es','pt']
    #nonlatin_languages = ['hi', 'cs', 'ja', 'zh-cn', 'vi']
    for target_language in latin_languages:
      translated = translator.translate('hi i am long',src='en', dest=target_language)
      paraphrase = translator.translate(translated.text, src=target_language, dest='en')
      # the below line adds a single new augmented example to the dataset. 
      # note that the guid should be a unique ID for this example, so you'll want to vary this
      # depending on how you generate your paraphrases
      train_examples_augmented.append(paraphrase.text)

AttributeError: ignored

In [6]:
data_dir = '/dataset'
task_processor = glue_processors[f"{task_name.lower()}-2"]()
train_examples = task_processor.get_train_examples(data_dir)

train_examples_augmented = []

### (incomplete) list of languages you can use
languages = [
    'en', # english
    'cs',  # czech
    'de',  # german
    'es', # spanish
    'fi',  # finnish
    'fr', # french
    'hi', # hindi
    'it', # italian
    'ja', # japanese
    'pt', # portuguese
    'ru', # russian
    'vi', # vietnamese
    'zh-cn',  # chinese
    ]

# generate some augmented examples for each training example
for example in train_examples:
    train_examples_augmented.append(example) # always include the original example
    latin_languages = ['de', 'fr', 'it', 'es','pt']
    #nonlatin_languages = ['hi', 'cs', 'ja', 'zh-cn', 'vi']
    for target_language in latin_languages:
      translated = translator.translate(example.text_a, src='en', dest=target_language)
      paraphrase = translator.translate(translated.text, src=target_language, dest='en')
      # the below line adds a single new augmented example to the dataset. 
      # note that the guid should be a unique ID for this example, so you'll want to vary this
      # depending on how you generate your paraphrases
      train_examples_augmented.append(InputExample(guid=f"{example.guid}-aug-{target_language}",
                                                      text_a=paraphrase.text,
                                                      text_b=None,
                                                      label=example.label))
output_dir = f"./data/tiny{task_name}-bt"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
with open(os.path.join(output_dir, "train.tsv"), "w") as writer:
    writer.write("sentence\tlabel\n")
    for example in train_examples_augmented:
        writer.write(f"{example.text_a}\t{example.label}\n")
tsv_to_csv(os.path.join(output_dir, "train.tsv"), os.path.join(output_dir, "train.csv"))

# Copy the original tinySST's dev set to the new directory
import shutil
shutil.copyfile(f"{data_dir}/dev.csv", f"{output_dir}/dev.csv")

NameError: ignored